In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import src.config as config

In [3]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=70) #28

current_date=Timestamp('2024-03-25 20:00:00')


In [4]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # # download 2 files from website
    # rides = load_raw_data(year=from_date_.year) #, months=from_date_.month)
    # rides = rides[(rides.pickup_datetime >= from_date_) & (rides.pickup_datetime < to_date_)]
   
    # rides_2 = load_raw_data(year=to_date_.year) #, months=to_date_.month)
    # rides_2 = rides_2[(rides_2.pickup_datetime < to_date_) & (rides_2.pickup_datetime < to_date_)]

    # rides = pd.concat([rides, rides_2]) 

    # Intenta cargar los datos del primer año
    rides = load_raw_data(year=from_date_.year)
    rides = rides[(rides.pickup_datetime >= from_date_) & (rides.pickup_datetime < to_date_)]

    # Verifica si los años son diferentes
    if from_date_.year != to_date_.year:
    # Carga los datos del segundo año
        rides_2 = load_raw_data(year=to_date_.year)
        rides_2 = rides_2[(rides_2.pickup_datetime >= from_date_) & (rides_2.pickup_datetime < to_date_)]
        rides = pd.concat([rides, rides_2]) 


    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [5]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2024-01-15 20:00:00'), to_date_=Timestamp('2023-03-27 20:00:00')
File 2023 was already in local storage


In [6]:
len(rides.pickup_location_id.unique())

320

In [7]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 320/320 [00:01<00:00, 209.70it/s]


In [8]:
rides.head()

,pickup_datetime,pickup_location_id,pickup_hour
362290,2024-01-16 02:55:14,2,2024-01-16 02:00:00
1419802,2024-01-16 06:31:31,2,2024-01-16 06:00:00
1553801,2024-01-16 07:03:57,2,2024-01-16 07:00:00
1146151,2024-01-16 07:36:16,2,2024-01-16 07:00:00
809036,2024-01-16 10:36:17,2,2024-01-16 10:00:00


In [9]:
ts_data.tail()

,pickup_hour,rides,pickup_location_id
537595,2024-03-25 15:00:00,0,469
537596,2024-03-25 16:00:00,2,469
537597,2024-03-25 17:00:00,0,469
537598,2024-03-25 18:00:00,0,469
537599,2024-03-25 19:00:00,0,469


In [10]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/100501
Connected. Call `.close()` to terminate connection gracefully.


In [11]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

Uploading Dataframe: 0.00% |          | Rows 0/537600 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/100501/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1523a6badf0>, None)